In [ ]:
import basicRNAseqAnalysis_imports

In [ ]:
#!git mv skymap_widget_imports.py basicRNAseqAnalysis_imports.py 

# Search and compare RNA-seq profiles based on experimental conditions

| Example comparison query | description|
|---|---|
| T-Cell, B-Cell | Differential expression analysis between profiles with annotation "T-Cell" and "B-Cell"|
| single.\*cell.\*neuron, single.\*cell.\*glioblastoma | Differential expression analysis between profiles with annotation "single cell neuron" and "single cell glioblastoma"|

Query format: Each query is a list of regulary expressions deliminated by a comma, where each regular expression define a group in the comparison. 

[Click here for more info on SkyMap](./README.ipynb)

In [ ]:
display(skymap_widget_imports.accordion)
display(skymap_widget_imports.widget_specie)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>


### Example analysis: Simple differential expression analysis between the thousands of human B-Cells and T-Lymphocytes expression profiles  



This notebook is a template which consist of the following steps:
1. Setting input free-text regex queries to define the classes of experimental conditions (queryLabelToRegexDict)
2. Querying the reprocessed data which consist of >400,000 expression profiles
3. Generate a fully annotated expression matrix
4. DE analysis like volcano plot, correlation heatmap and PCA. (Most plots are interactive, u can download with a simple click)

### More parameters: 

expression_metric: Those are Kallisto expression metric: "tpm","est_counts"
baseDir: if run locally, change it to mirror our path.

In [ ]:
from  skymap_widget_imports import *
import re

In [ ]:
expression_metric='tpm' #

In [ ]:
querySpecie=skymap_widget_imports.widget_specie.get_interact_value()

In [ ]:
queryStr=skymap_widget_imports.widget_query.get_interact_value()

listOfQueries=re.split(" *, *", queryStr)

if len(queryStr)<5:
    raise ValueError('Please provide a query with more than 5 characters')
if len(listOfQueries)<2:
    raise ValueError('Please provide a query with more than 2 conditions')


In [ ]:
queryLabelToRegexDict=dict(zip(listOfQueries,listOfQueries))

# Data loading

### load in SRS biospecieman annotations

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np

allSRS_pickle_dir='/home/jovyan/efs/all_seq/meta_data/allSRS.with_processed_data.flat.pickle.gz'
%time allSRS=pd.read_pickle(allSRS_pickle_dir)
allSRS.index.names=['SRS']

### load in technical metadata

In [ ]:
sra_dump_pickle_dir='/home/jovyan/efs/all_seq/meta_data/sra_dump.fastqc.bowtie_algn.pickle'
%time technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)
technical_meta_data_df[('SRAmeta','Run')]=technical_meta_data_df.index

### load the expression matrix

Check files in baseDir directory for more species

In [ ]:
def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf
data_matrix_dir=baseDir+'/{specie}.gene_symbol.{expression_metric}'.format(specie=querySpecie,
                                            expression_metric=expression_metric)

%time rnaseqDf=loadDf(data_matrix_dir)

# Find the relevent SRS (Sample  IDs)  


In [ ]:
myL=[]
for  queryRegex in queryLabelToRegexDict.values():
    %time hitSrsS=allSRS[allSRS.str.contains(queryRegex,case=False)]
    myL.append(hitSrsS)

queryLabel='queryLabel'
mergeS=pd.concat(myL,keys=queryLabelToRegexDict.keys(),names=[queryLabel])
mergeS_noDup=mergeS.groupby(['SRS','queryLabel']).first()
unqiueHitMask=mergeS_noDup.groupby('SRS').size()==1
unqiueHitSrs=unqiueHitMask.index[unqiueHitMask]
mergeS_noDup_unique=mergeS_noDup[mergeS_noDup.index.get_level_values('SRS').isin(unqiueHitSrs)]

Number of SRS per query class

In [ ]:
mergeS_noDup_unique.groupby(queryLabel).size()

In [ ]:
srsToClasses_all=mergeS_noDup_unique.reset_index().set_index(['SRS'])['queryLabel']

srsToClasses=srsToClasses_all

### map SRS Ids to SRR Ids

In [ ]:
m_SRAMeta=technical_meta_data_df[('SRAmeta','Sample')].isin(srsToClasses.index)
technical_meta_data_df_hit=technical_meta_data_df[m_SRAMeta]

SRAMetasrsCorrespondingQuery=srsToClasses.loc[technical_meta_data_df_hit[('SRAmeta','Sample')]].values
technical_meta_data_df_hit[('SRAmeta',queryLabel)]=SRAMetasrsCorrespondingQuery
relevantMetaColsL=[('SRAmeta',queryLabel),('SRAmeta','Study'),('SRAmeta','Sample'),('SRAmeta','Run'),('SRAmeta','ScientificName')]
technical_meta_data_df_sub=technical_meta_data_df_hit[relevantMetaColsL]
designDf=technical_meta_data_df_sub['SRAmeta']

Top species with reprocessed data

In [ ]:
designDf.groupby(['queryLabel','ScientificName']).size()

In [ ]:
hitSrsAllAnnotS=allSRS[alxlSRS.index.get_level_values('SRS').isin(mergeS.index.get_level_values('SRS'))]

In [ ]:
srsToTextS=hitSrsAllAnnotS

In [ ]:
srsToTextS=pd.Series(data="NCBI SRA SRS:"+srsToTextS.index+' <br> '+srsToTextS.values,index=srsToTextS.index)

In [ ]:
designDf['Description']=srsToTextS[designDf.Sample].values

In [ ]:
%time designDf_specie=designDf[(designDf['ScientificName']==querySpecie)&(designDf.Run.isin(rnaseqDf.columns))]
queryDesignDf=designDf_specie

Number of samples per query class with data

In [ ]:
designDf_specie.groupby(queryLabel).size()

In [ ]:
%time hitDf=pd.DataFrame( list(map( lambda srrId: rnaseqDf[srrId],queryDesignDf.Run))).T
hitDf.columns=queryDesignDf.set_index(queryDesignDf.columns.tolist()).index

### Output:  fully annnotated matrix matrix

Example layout is listed in the cell below

In [ ]:
hitDf.head()

#  Example analysis with output matrix


In [ ]:
inputAnalyzeDf=np.log2(hitDf+1)

### Interactive PCA 2D
to show that samples are grouped in the higher dimensional space according to the class

In [ ]:
#plotPcs=[0,1]

inPcaDf=inputAnalyzeDf.T

from sklearn import decomposition
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()


PCA=decomposition.PCA(n_components=3)
pcaM=PCA.fit_transform((inPcaDf))
pcaDf=pd.DataFrame( data=pcaM,index=inPcaDf.index)

layout = go.Layout(
    yaxis={'title':"PC1"},
    xaxis={'title':"PC0"}
)

fig = go.Figure(layout=layout)
for label, sub_pca_df in pcaDf.groupby('queryLabel'):
    fig.add_scatter(x=sub_pca_df[0], y=sub_pca_df[1],
                      name=label,
                    hovertext=sub_pca_df.index.get_level_values('Description'),
                    mode = 'markers')
iplot(fig)

### Interactive PCA 3D

Sometimes the first PC is picking up just the batch effect. 3D visualization makes it lot easier to see the seperation

In [ ]:
layout_3d = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='PC0'),
                    yaxis = dict(
                        title='PC1',),
                    zaxis = dict(
                        title='PC2',),),
                  )

fig = go.Figure(layout=layout_3d)
for label, sub_pca_df in pcaDf.groupby('queryLabel'):
    fig.add_scatter3d(x=sub_pca_df[0], y=sub_pca_df[1],z=sub_pca_df[2],
                      name=label,
                    hovertext=sub_pca_df.index.get_level_values('Description'),
                    mode = 'markers')
iplot(fig)

### Interactive TSNE

In [ ]:
from sklearn import manifold
madS=inPcaDf.mad(axis=0).sort_values()
genesWithVarianceL=madS.iloc[-1500:].index
withDeviationDf=inPcaDf.loc[:,genesWithVarianceL]

TSNE=manifold.TSNE(n_components=2)
%time tsneM=TSNE.fit_transform((withDeviationDf))
tsneDf=pd.DataFrame( data=tsneM,index=inPcaDf.index)

fig = go.Figure()
for label, sub_df in tsneDf.groupby('queryLabel'):
    fig.add_scatter(x=sub_df[0], y=sub_df[1] ,name=label,
                    hovertext=sub_df.index.get_level_values('Description'),
                    mode = 'markers')
iplot(fig)

# Volcano plot with t-test

To edit the query conditions to be compared in volcano plot:
1. Change the group labels in the drop down widget below
* Click on next next cell (Anywhere is fine), the cell that starts with "from scipy import stats"

* Hit the "Run" button from the toolbar. 


In [ ]:
wA=widgets.Dropdown(
    options=listOfQueries,
    value=listOfQueries[0],
    description='condition A:',
    disabled=False,
)
wB=widgets.Dropdown(
    options=listOfQueries,
    value=listOfQueries[1],
    description='condition B:',
    disabled=False,
)

display(wA)
display(wB)

In [ ]:
from scipy import stats
expression_effect_size_filter=0.0 
labelA=wA.get_interact_value()
labelB=wB.get_interact_value()
t,p=stats.ttest_ind(inputAnalyzeDf[labelA],inputAnalyzeDf[labelB],axis=1)
effectDiff=inputAnalyzeDf[labelA].mean(axis=1)-inputAnalyzeDf[labelB].mean(axis=1)
effectLabel='expression of : "{}" - "{}"'.format(labelA, labelB)
tmpDf=pd.DataFrame({'t':t,'-log10(p)':-np.log10(p),effectLabel:effectDiff,'u':inputAnalyzeDf.mean(axis=1)},index=inputAnalyzeDf.index)
plotDf=tmpDf[tmpDf['u']>=expression_effect_size_filter].dropna()
yLabel='-log10(p)'
xLabel=effectLabel


layout = go.Layout(
    yaxis={'title':"-log10(p)"},
    xaxis={'title':"{} - {}".format(labelA,labelB)}
)
fig = go.Figure(layout=layout)
#m=plotDf.index.isin(annotateGenesList)
fig.add_scatter( x=plotDf[xLabel][~m],y=plotDf[yLabel][~m],mode='markers',hovertext=plotDf.index.values,name='')
#fig.add_scatter( x=plotDf[xLabel][m],y=plotDf[yLabel][m],mode='markers+text',text=plotDf[m].index.values,
#                textposition='bottom center',
#                name='Query annotation gene')
iplot(fig)

### Boxplot showing the expression level of a gene

Feel free to change the annotateGenesList to something else

In [ ]:
annotateGenesList=['GAPDH','TP53']#np.random.choice(inputAnalyzeDf.index,2) #['GAPDH','TP53']

print ('Genes plotting: ',annotateGenesList)

layout = go.Layout(
    boxmode='group',
yaxis={'title':"Expression level of gene measured in log2 "+expression_metric.upper()}
)

tmpDf=inputAnalyzeDf.loc[annotateGenesList].T.stack()#.reset_index()

tmpDf2=tmpDf.reset_index(name='Expression')

myL=[]
for myQueryLabel,tmpDf3 in tmpDf2.groupby('queryLabel'):
    myL.append(go.Box(name=myQueryLabel,y=tmpDf3['Expression'],x=tmpDf3['level_6']))
    
fig=go.Figure(data=myL,layout=layout)
iplot(fig )

### Study level condition correlation heatmap

Useful for finding outliers

In [ ]:
%matplotlib inline
myColorL=['red','blue','green','orange','pink'] ##add more colors as you see fit
sigDf=inputAnalyzeDf.groupby(level=['queryLabel','Study'],axis=1).mean()
corrDf=sigDf[sigDf.mad(axis=1)>np.log2(5)].corr()
corrDf_valid=corrDf.loc[~corrDf.isnull().all(axis=1),~corrDf.isnull().all(axis=0)]
labelToColorS=pd.Series(dict(zip(corrDf_valid.index.get_level_values('queryLabel').unique(),myColorL)))
print (labelToColorS)
colors=labelToColorS[corrDf_valid.index.get_level_values('queryLabel')].values
g=sns.clustermap(data=corrDf_valid,col_colors=colors,metric='euclidean',annot=True)
g

# Content page for completed results

* [Output: fully annnotated matrix matrix](#Output:--fully-annnotated-matrix-matrix)
* [Interactive PCA 2D](#Interactive-PCA-2D)
* [Interactive PCA 3D](#Interactive-PCA-3D)
* [Interactive TSNE](#Interactive-TSNE)
* [Volcano plot with t-test](#Volcano-plot-with-t-test)
* [Boxplot showing the expression level of a gene ](#Boxplot-showing-the-expression-level-of-a-gene)
* [Study level condition correlation heatmap](#Study-level-condition-correlation-heatmap)